In [1]:
import pandas as pd
from ptrail.core.TrajectoryDF import PTRAILDataFrame
from ptrail.visualization.TrajPlotter import TrajectoryPlotter as tpl
from ptrail.visualization.statViz import StatViz as stv
from ptrail.visualization.InteractiveDonut import InteractiveDonut as donut
import plotly.express as px

# Import the starkey trajectory dataset.
pdf = pd.read_csv('./starkey.csv')
starkey = PTRAILDataFrame(data_set=pdf,
                          latitude='lat',
                          longitude='lon',
                          datetime='DateTime',
                          traj_id='Id')
starkey.head(5)

lat         lon  StarkeyTime    GMDate  \
traj_id   DateTime                                                            
880109D01 1995-04-13 13:40:06  45.239682 -118.533204    229902006  21:40:06   
          1995-04-15 12:16:15  45.250521 -118.530438    230069775  20:16:15   
          1995-04-15 21:39:38  45.247943 -118.541455    230103578  05:39:38   
          1995-04-16 03:32:14  45.247429 -118.539530    230124734  11:32:14   
          1995-04-16 04:08:28  45.247117 -118.542579    230126908  12:08:28   

                                 GMTime   LocDate   LocTime  RadNum Species  \
traj_id   DateTime                                                            
880109D01 1995-04-13 13:40:06  19950413  19950413  13:40:06     409       D   
          1995-04-15 12:16:15  19950415  19950415  12:16:15     409       D   
          1995-04-15 21:39:38  19950416  19950415  21:39:38     409       D   
          1995-04-16 03:32:14  19950416  19950416  03:32:14     409       D   
          1995-04-16 04:08:28  19950416  19950416  04:08:28     409       D   

                                 UTME     UTMN  Year  Grensunr  Grensuns  \
traj_id   DateTime                                                         
880109D01 1995-04-13 13:40:06  379662  5010734    95  13:13:00  02:39:00   
          1995-04-15 12:16:15  379895  5011927    95  13:09:00  02:41:00   
          1995-04-15 21:39:38  379039  5011656    95  13:07:00  02:43:00   
          1995-04-16 03:32:14  379188  5011581    95  13:07:00  02:43:00   
          1995-04-16 04:08:28  378938  5011567    95  13:07:00  02:43:00   

                               Obswt  
traj_id   DateTime                    
880109D01 1995-04-13 13:40:06   1.47  
          1995-04-15 12:16:15   1.59  
          1995-04-15 21:39:38   1.34  
          1995-04-16 03:32:14   1.50  
          1995-04-16 04:08:28   1.34

In [2]:
# Import the starkey habitat datset.
starkey_habitat = pd.read_csv('./starkey_habitat.csv')
starkey_habitat.head()

,lat,lon,SoilDpth,PerSlope,SINAspct,COSAspct,Convex3,DistCWat,Canopy,Elev,DistEWat,EcoGener,DistOPEN,DistRSTR,DistCLSD,DistEFnc,CowPast,ForgProd,DistEdge
0,45.272378,-118.610193,14,5,-0.81,0.59,500.63,218,3,1389,212,GB,127,2271,30,0,SMITH-BALLY,363,0
1,45.272648,-118.610201,14,5,-0.71,0.71,500.21,228,3,1388,218,GB,150,2293,30,0,SMITH-BALLY,363,0
2,45.272918,-118.610208,14,4,-0.89,0.45,499.93,242,3,1387,228,GB,170,2315,30,0,SMITH-BALLY,363,0
3,45.270224,-118.609750,14,8,-0.98,0.21,500.68,201,5,1393,201,GB,30,2081,134,0,SMITH-BALLY,363,0
4,45.270494,-118.609757,14,7,-0.97,0.22,501.22,190,5,1393,190,GB,30,2101,108,0,SMITH-BALLY,363,0


In [3]:
# Show the trajectory viewer.
# tpl.show_trajectories(starkey, opacity=0.75, weight=2)

In [4]:
# Show the treemap of the average daily distance
# of the animals.
# stv.trajectory_distance_treemap(dataset=starkey,
#                                 path=[px.Constant('All'), 'Species','traj_id'])

In [5]:
# Plot the donut plot which shows how starkey forest
# area is divided into several pastures.
# donut.plot_area_donut(habitat=starkey_habitat)

In [6]:
# plot the interactive visualization showing the distribution of
# animals by pasture.
# donut.animals_by_pasture(trajectories=starkey, habitat=starkey_habitat)

In [7]:
from ptrail.preprocessing.filters import Filters as filt
from ptrail.features.temporal_features import TemporalFeatures as temp
from ptrail.utilities.DistanceCalculator import FormulaLog as calc
pd.options.mode.chained_assignment = None

starkey = temp.create_date_column(starkey)
a = starkey_habitat[(starkey_habitat['EcoGener'] == 'WR')]
waterbodies = a.loc[(a['DistEWat'] == 0)]

bboxes = []
for i in range(len(waterbodies)):
    lat = waterbodies.iloc[i]['lat']
    lon = waterbodies.iloc[i]['lon']
    bboxes.append(filt.get_bounding_box_by_radius(lat, lon, 1000))

waterbodies['bbox'] = bboxes

In [8]:


chunks = []
for i in range(len(waterbodies)):
    chunks.append(
        filt.filter_by_bounding_box(starkey, waterbodies.iloc[i]['bbox'])
    )

In [14]:
dataset = []
for i in range(len(chunks)):
    small_df = pd.DataFrame(columns=['traj_id', 'days']).set_index('traj_id')
    ids_ = chunks[i].reset_index()['traj_id'].unique()
    for j in range(len(ids_)):
        filt = chunks[i].reset_index().loc[chunks[i].reset_index()['traj_id'] == ids_[j]]
        small_df.loc[ids_[j]] = pd.to_datetime(filt.Date).nunique()
    print(len(small_df))
    dataset.append(small_df)

183
184
173
173
174
176
178
184
183
168
173
173
173
173
183
168
168
168
171
171
172
172
183
183
185
185
183
181
179
177
178
176
175
173
173
169
169
167
167
167
168
168
167
167
165
165
164
160
160
162
163
160
160
158
158
158
158
157
153
152
152
155
155
154
156
156
156
155
149
150
149
138
138
136
133
132
131
128
121
121
120
121
119
115
112
112
112
112
111
110
110
